In [1]:
import argparse
import logging
import time
import sys
import cv2
import numpy as np
import pickle
import random

from tf_pose.estimator import BodyPart
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

/Users/david/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
logger = logging.getLogger('TfPoseEstimator-Video')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

fps_time = 0

In [3]:
def process_human_data(humans):
    
    if (len(humans)==0):
        return np.zeros(shape=(18,2))
                        
    feature = np.zeros(shape=(18,2))
    for i in range(18):
        if i not in humans[0].body_parts:
            feature[i] = [0, 0]
        else:
            feature[i] = [humans[0].body_parts[i].x, humans[0].body_parts[i].y]
    
    return feature
        

In [4]:
def inference_video(path):
    model_path='mobilenet_thin'
    resolution = '320x240'
    showBG=True

    logger.debug('initialization %s : %s' % (model_path, get_graph_path(model_path)))
    w, h = model_wh(resolution)
    e = TfPoseEstimator(get_graph_path(model_path), target_size=(w, h))


    cap = cv2.VideoCapture(path)

    #---------------modified----------------#
    num_frames = float(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #print ("All Frames: " ,num_frames)
    cur_frames = 0.0
    step = (num_frames / 20.0) 
    #---------------modified----------------#

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    resize_out_ratio = 8.0
    #print("Image Size: %d x %d" % (width, height)) 

    single_video_features = np.array([])
    if cap.isOpened() is False:
        print("Error opening video stream or file")

    while (cap.isOpened()):   
        if(cur_frames >= num_frames):
            break

        frame_no = (cur_frames/num_frames)
        cap.set(1,frame_no)
        ret_val, image = cap.read()

        #print("Frame no: ", frame_no)
        #print ("Count: ", cur_frames)

        if ret_val == True:
            humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=resize_out_ratio)
            #print ("Frame numbers: ", cur_frames, humans)
            frame_feature = process_human_data(humans) 
            single_video_features = np.append(single_video_features, frame_feature) 

        cur_frames+=step
        if cv2.waitKey(1) == 27:
            break
    #print (single_video_features)
    cv2.destroyAllWindows()
    #logger.debug('finished+')
    return single_video_features
    

In [5]:
def get_classification(filename):
    label=np.zeros(shape=(9))
    label[video_dict[filename]]=1
    return label

In [ ]:
import os
from pathlib import Path

video_dict = { 'PlayingCello':0,'PlayingDaf':1,
              'PlayingDhol':2,'PlayingFlute':3,
              'PlayingGuitar':4,'PlayingPiano':5
              , 'PlayingSitar':6,'PlayingTabla':7,'PlayingViolin':8}

mypath = Path().absolute()
dataset_path = os.path.abspath(os.path.join(mypath, os.pardir))+"/action_dataset"

feature_set=[]

for subdir, dirs, files in os.walk(dataset_path):
    for dirss in dirs:
        finished_video = 0
        if (dirss in video_dict):
            for filename in os.listdir(os.path.join(subdir,dirss)):
                abs_path =os.path.join(subdir,dirss,filename)
                feature =inference_video(abs_path)
                classification = get_classification(dirss)
                feature =list(feature)
                feature_set.append([feature,classification])
                print("Finish video: ", finished_video+1)
                finished_video+=1
with open('feature_set.pickle','wb') as file:
    pickle.dump(feature_set,file)


[2018-07-25 17:16:09,473] [TfPoseEstimator-Video] [DEBUG] initialization mobilenet_thin : /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb
[2018-07-25 17:16:09,474] [TfPoseEstimator] [INFO] loading graph from /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb(default size=320x240)
[2018-07-25 17:16:31,648] [TfPoseEstimator-Video] [DEBUG] initialization mobilenet_thin : /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb
[2018-07-25 17:16:31,648] [TfPoseEstimator] [INFO] loading graph from /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb(default size=320x240)


Finish video:  1


[2018-07-25 17:16:49,401] [TfPoseEstimator-Video] [DEBUG] initialization mobilenet_thin : /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb
[2018-07-25 17:16:49,402] [TfPoseEstimator] [INFO] loading graph from /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb(default size=320x240)


Finish video:  2


[2018-07-25 17:17:08,363] [TfPoseEstimator-Video] [DEBUG] initialization mobilenet_thin : /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb
[2018-07-25 17:17:08,364] [TfPoseEstimator] [INFO] loading graph from /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb(default size=320x240)


...

Finish video:  54


[2018-07-25 18:30:13,367] [TfPoseEstimator-Video] [DEBUG] initialization mobilenet_thin : /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb
[2018-07-25 18:30:13,371] [TfPoseEstimator] [INFO] loading graph from /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb(default size=320x240)


Finish video:  55


[2018-07-25 18:34:44,100] [TfPoseEstimator-Video] [DEBUG] initialization mobilenet_thin : /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb
[2018-07-25 18:34:44,105] [TfPoseEstimator] [INFO] loading graph from /Users/david/Documents/system_implemetation/tf_action_recognition/models/graph/mobilenet_thin/graph_opt.pb(default size=320x240)


Finish video:  56


In [ ]:
import pickle
import random

def load_dataset_and_label:
    with open('feature_set.pickle','rb') as f:
		features_set = pickle.load(f)
        random.shuffle(features_set)
        features_set = np.array(features_set)
        
        testing_size_rate = 0.1
        testing_size = int(testing_size_rate*len(features_set))
        
        
        